In [1]:
u = garner.Auth('kipsterbroz@gmail.com', 'dbfaab17-b8b1-41ad-95ec-1568ac734109')

NameError: name 'garner' is not defined

In [1]:
import garner

ParamValidationError: Parameter validation failed:
Invalid type for parameter Logins.cognito-idp.eu-west-2.amazonaws.com/eu-west-2_ewIawpZwb, value: None, type: <class 'NoneType'>, valid types: <class 'str'>

In [3]:
garner.login("kipgparker@gmail.com")

Enter Password


Password: ·········


Authenticated


In [4]:
garner.api

<module 'garner.api' from 'C:\\Users\\kipst\\OneDrive\\Desktop\\GarnerPython\\garner\\api\\__init__.py'>

In [6]:
file_uploader = CognitoUserFileUploader(
    id_token = u.id_token
)
file_uploader.upload_file('movie.gif')

In [7]:
import requests
import json

APPSYNC_API_ENDPOINT_URL = config["aws_appsync_graphqlEndpoint"]

headers = {
    'Authorization': str(u.access_token)
}

params = {
    "filter": {"privateKey" : {"eq": u.private_key}}
}

def execute_gql(query):
    u.check_token()
    payload_obj = {"query": query, "variables" : params,}
    payload = json.dumps(payload_obj)
    response = requests.request("POST", APPSYNC_API_ENDPOINT_URL, data=payload, headers=headers)
    return response

query = """query ListPools(
    $filter: ModelPoolFilterInput
    $limit: Int
    $nextToken: String
  ) {
    listPools(filter: $filter, limit: $limit, nextToken: $nextToken) {
      items {
        id
        title
        privateKey
      }
    }
  }"""

output = execute_gql(query).json()

#print(output)
print("working on: {}".format('\033[1m' +output['data']['listPools']['items'][0]['title']))
#for item in output['data']['listPools']['items']:
   # print("-{}".format(item['title'][]))

working on: kai man
